In [144]:
import pandas as pd
import numpy as np
import folium
import plotly.express as px
import plotly.graph_objects as go
import datetime
import requests

In [145]:
df = pd.read_csv('Locatievoorstellen.csv', header=1, index_col=0)

In [146]:
pd.set_option('display.max_columns', None)

In [147]:
df.columns = df.columns.str.replace(' - ', '_')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

In [148]:
df["tijdstip_van_laatste_wijziging"] = pd.to_datetime(df["tijdstip_van_laatste_wijziging"])
df["tijdstip_aangemaakt"] = pd.to_datetime(df["tijdstip_aangemaakt"])
df["tijdstip_locatievoorstel"] = pd.to_datetime(df["tijdstip_locatievoorstel"])
df["tijdstip_controle_locatievoorstel"] = pd.to_datetime(df["tijdstip_controle_locatievoorstel"])
df["tijdstip_advies_exploitant"] = pd.to_datetime(df["tijdstip_advies_exploitant"])
df["tijdstip_publicatie_verkeersbesluit"] = pd.to_datetime(df["tijdstip_publicatie_verkeersbesluit"])
df["tijdstip_aanvraag_netaansluiting"] = pd.to_datetime(df["tijdstip_aanvraag_netaansluiting"])
df["tijdstip_definitief_verkeersbesluit"] = pd.to_datetime(df["tijdstip_definitief_verkeersbesluit"])
df["tijdstip_accepteren_mra-e"] = pd.to_datetime(df["tijdstip_accepteren_mra-e"])
df["tijdstip_realisatie_plannen"] = pd.to_datetime(df["tijdstip_realisatie_plannen"])
df["tijdstip_locatie_voorbereiden"] = pd.to_datetime(df["tijdstip_locatie_voorbereiden"])
df["tijdstip_in_bedrijf"] = pd.to_datetime(df["tijdstip_in_bedrijf"])
df["tijdstip_opleveren_laadpaal"] = pd.to_datetime(df["tijdstip_opleveren_laadpaal"])
df["tijdstip_opleveren_locatie"] = pd.to_datetime(df["tijdstip_opleveren_locatie"])

In [149]:
df['doorlooptijd_d'] = (df['tijdstip_opleveren_locatie'] - df['tijdstip_aangemaakt'])

In [150]:
df = df.drop(columns=['stilgezet_uitleg','verplaatsing_mra-e_laadpaal_id', 'opmerkingen', 'toelichting_bij_verplaatsing', 
                      'locatiecode', 'aanbodgestuurd_opmerkingen', 'datagestuurd_opmerkingen', 'gemeente_akkoord_met_offerte_verplaatsing',
                     'extra_laadpaal_ids', 'referenties', 'aanleiding'])

In [151]:
df['postcode'] = df['postcode'].str.replace('[a-zA-Z]', '', regex=True)
df['postcode'] = df['postcode'].str.replace(' ', '', regex=True)

In [152]:
df.loc[(df['stilgezet'] == False) & df['doorlooptijd_d'].notna(), ['exploitant_akkoord', 'aannemer_akkoord', 'gemeente_akkoord']] = True
df.loc[(df['stilgezet'] == True) | df['doorlooptijd_d'].isna(), ['exploitant_akkoord', 'aannemer_akkoord', 'gemeente_akkoord']] = False

In [164]:
df['datum_oplevering_y'] = pd.DatetimeIndex(df['datum_oplevering']).year
df['datum_oplevering_y'] = df['datum_oplevering_y'].fillna(0).astype(int)
df.loc[(df['datum_oplevering_y'] == 0), 'datum_oplevering_y'] = None

In [154]:
df = df.loc[(df['datum_oplevering_y'] < 2024)]

In [159]:
df.head()

,locatievoorstel_id,stilgezet,stilgezet_nu_of_ooit_in_verleden,tijdstip_van_laatste_wijziging,tijdstip_aangemaakt,tijdstip_locatievoorstel,tijdstip_controle_locatievoorstel,tijdstip_advies_exploitant,tijdstip_publicatie_verkeersbesluit,tijdstip_aanvraag_netaansluiting,tijdstip_definitief_verkeersbesluit,tijdstip_accepteren_mra-e,tijdstip_realisatie_plannen,tijdstip_locatie_voorbereiden,tijdstip_in_bedrijf,tijdstip_opleveren_laadpaal,tijdstip_opleveren_locatie,organisatie_id,organisatie_naam,gemeente_id,gemeente_naam,locatie_longitude,locatie_latitude,straat,huisnummer,postcode,plaats,was_voorkeurslocatie,voorkeurslocatieid,autodelen,laadplein,laadpunten,gereserveerde_parkeervakken,gereserveerde_parkeervakken_autodelen,exploitant_akkoord,aannemer_akkoord,gemeente_akkoord,verkeersbesluit_gepubliceerd_datum,verkeersbesluit_definitief_datum,verkeersbesluit_definitief_gewijzigd,aanvraag_netaansluiting_datum,datum_realisatie_gepland,datum_in_bedrijf,laadpaal_id,datum_oplevering,doorlooptijd_d,datum_oplevering_y
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LV-6,False,False,2020-10-22 09:06:27.400,2018-12-05 13:40:44.360,2018-12-05 14:06:07.114,2018-12-19 14:45:00.395,2018-12-24 14:47:22.368,2018-12-31 08:50:02.188,2019-01-31 13:11:05.446,2018-12-31 10:46:40.870,2018-12-31 10:46:44.903,2019-04-24 05:54:56.583,2019-04-24 06:36:17.898,2019-05-08 08:36:20.072,2019-08-14 08:56:36.783,2019-09-11 07:24:17.680,84,OVER-Gemeenten,385,Wormerland,4.810485,52.494985,Spatterstraat,21,1531,Wormer,False,<nil>,False,False,2,2,0,True,True,True,2018-08-28 00:00:00,2018-10-09 00:00:00,False,2019-01-21 00:00:00,2019-05-06 00:00:00,2019-05-07 00:00:00,PP001911,2019-05-08 00:00:00,279 days 17:43:33.320000,2019
2,LV-8,False,False,2020-10-22 09:06:27.400,2018-12-06 09:20:44.736,2018-12-31 12:39:10.967,2018-12-31 12:39:30.972,2019-01-02 08:06:41.882,2019-02-25 15:19:54.254,2019-03-07 06:43:14.800,2019-04-09 07:55:49.334,2019-04-09 08:45:33.427,2019-09-11 06:55:01.214,2019-09-25 12:08:00.393,2019-09-25 08:24:13.355,2019-10-17 14:27:06.405,2019-09-25 12:08:18.781,16,Gemeente De Bilt,269,De Bilt,5.180691,52.111028,Buys Ballotweg,66,3731,De Bilt,False,<nil>,False,False,2,2,0,True,True,True,2019-02-27 00:00:00,2019-04-10 00:00:00,False,2019-03-06 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,PP001968,2019-09-25 00:00:00,293 days 02:47:34.045000,2019
3,LV-11,False,False,2020-10-22 09:06:27.400,2018-12-07 09:59:17.463,2018-12-07 10:03:25.510,2019-01-14 12:10:51.520,2019-01-15 08:45:00.559,2019-01-31 15:28:01.963,2019-02-18 14:02:10.212,2019-02-20 07:31:18.342,2019-02-20 11:24:01.657,2019-06-07 06:57:55.205,2019-06-11 06:57:01.107,2019-06-13 12:08:19.299,2019-08-14 08:58:25.607,2020-07-16 14:50:16.003,28,Gemeente Heemskerk,334,Heemskerk,4.655286,52.511481,Commandeurslaan,81,1964,Heemskerk,False,<nil>,False,False,2,2,0,True,True,True,2019-01-25 00:00:00,2019-02-27 00:00:00,False,2019-02-14 00:00:00,2019-06-11 00:00:00,2019-06-13 00:00:00,PP002024,2019-06-13 00:00:00,587 days 04:50:58.540000,2019
4,LV-12,False,False,2020-10-22 09:06:27.400,2018-12-07 10:17:23.991,2018-12-07 10:18:44.540,2018-12-19 14:45:58.376,2018-12-24 14:49:50.718,2019-01-31 15:33:03.486,2019-02-18 14:04:13.754,2019-02-20 07:25:32.040,2019-02-20 11:24:13.567,2019-06-07 07:00:04.976,2019-06-11 06:59:11.801,2019-06-13 12:07:04.829,2019-08-14 09:00:13.784,2020-07-16 14:55:31.495,28,Gemeente Heemskerk,334,Heemskerk,4.670446,52.508710,Maerten van Heemskerckstraat,25,1961,Heemskerk,False,<nil>,False,False,2,2,0,True,True,True,2019-01-18 00:00:00,2019-02-20 00:00:00,False,2019-02-14 00:00:00,2019-06-12 00:00:00,2019-06-13 00:00:00,PP002023,2019-06-13 00:00:00,587 days 04:38:07.504000,2019
5,LV-13,False,False,2020-10-22 09:06:27.400,2018-12-11 12:38:02.088,2019-01-31 15:45:37.668,2019-02-25 12:05:58.888,2019-02-25 13:45:57.918,2019-03-19 10:36:27.679,2019-03-25 08:51:28.078,2019-04-30 07:15:35.345,2019-04-30 07:26:09.665,2019-09-11 15:02:04.703,2020-01-27 12:13:29.181,2019-09-30 10:32:23.344,2020-0

Plotting 

In [163]:
fig = px.histogram(df, x='datum_oplevering_y')
fig.update_layout(title_text='Histogram of the number of locations per year', xaxis_title='Year', yaxis_title='Number of locations', legend_title='Was preferred location')
fig.show()

In [180]:
fig = px.histogram(df, x='stilgezet', color='stilgezet', histfunc='count')
fig.update_layout(title_text='Stopped and not stopped locations', xaxis_title='Has been stopped', yaxis_title='Number of locations', showlegend=False)
fig.update_xaxes(tickvals=[0, 1], ticktext=['No', 'Yes'])
fig.show()